# ✈️ Flight Operations Data Engineering Pipeline
## AviationStack + OpenSky Network Integration

This pipeline demonstrates **real-world data engineering skills** with heavy SQL usage:
- **Extract**: Live flight data from OpenSky Network API
- **Transform**: PySpark + Spark SQL for data processing
- **Load**: Star schema data warehouse design
- **Analyze**: SQL-based analytics and KPIs

### Why SQL Matters in Data Engineering
- 70%+ of data engineering work involves SQL
- Most data warehouses (Snowflake, BigQuery, Redshift) are SQL-based
- ETL/ELT pipelines heavily use SQL transformations

In [ ]:
# Cell 1: Setup and Installation
print("📦 Installing dependencies...")
!pip install -q pyspark requests pandas plotly

print("✅ Setup complete!")

In [ ]:
# Cell 2: Import Libraries
import requests
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import json
from datetime import datetime, timedelta
import time
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

print("✅ All libraries imported successfully!")

In [ ]:
# Cell 3: Initialize Spark Session with SQL Support
spark = SparkSession.builder \
    .appName("FlightOperationsPipeline") \
    .config("spark.sql.repl.eagerEval.enabled", True) \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.catalogImplementation", "hive") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

print(f"🚀 Spark version: {spark.version}")
print("✅ Spark SQL session initialized!")
print("\n📊 SQL is enabled - we'll use Spark SQL extensively in this pipeline!")

In [ ]:
# Cell 4: API Configuration
# OpenSky Network API - FREE, no key required for basic usage
OPENSKY_BASE_URL = "https://opensky-network.org/api"

# Define major airports for analysis (ICAO codes)
AIRPORTS = {
    'KJFK': {'name': 'John F. Kennedy International', 'city': 'New York', 'country': 'US', 'lat': 40.6413, 'lon': -73.7781},
    'KLAX': {'name': 'Los Angeles International', 'city': 'Los Angeles', 'country': 'US', 'lat': 33.9425, 'lon': -118.4081},
    'EGLL': {'name': 'London Heathrow', 'city': 'London', 'country': 'UK', 'lat': 51.4700, 'lon': -0.4543},
    'LFPG': {'name': 'Paris Charles de Gaulle', 'city': 'Paris', 'country': 'FR', 'lat': 49.0097, 'lon': 2.5479},
    'RJTT': {'name': 'Tokyo Haneda', 'city': 'Tokyo', 'country': 'JP', 'lat': 35.5494, 'lon': 139.7798},
    'WSSS': {'name': 'Singapore Changi', 'city': 'Singapore', 'country': 'SG', 'lat': 1.3644, 'lon': 103.9915},
    'OMDB': {'name': 'Dubai International', 'city': 'Dubai', 'country': 'AE', 'lat': 25.2528, 'lon': 55.3644},
    'VHHH': {'name': 'Hong Kong International', 'city': 'Hong Kong', 'country': 'HK', 'lat': 22.3080, 'lon': 113.9185},
    'EDDF': {'name': 'Frankfurt Airport', 'city': 'Frankfurt', 'country': 'DE', 'lat': 50.0379, 'lon': 8.5622},
    'YSSY': {'name': 'Sydney Kingsford Smith', 'city': 'Sydney', 'country': 'AU', 'lat': -33.9399, 'lon': 151.1753}
}

print(f"✅ Configured {len(AIRPORTS)} major international airports for analysis")

In [ ]:
# Cell 5: EXTRACT - Fetch Live Flight Data from OpenSky Network
def extract_flight_states():
    """Extract current flight states from OpenSky Network API"""
    
    url = f"{OPENSKY_BASE_URL}/states/all"
    
    print("📡 Fetching live flight data from OpenSky Network...")
    
    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        data = response.json()
        
        states = data.get('states', [])
        timestamp = data.get('time', int(datetime.now().timestamp()))
        
        print(f"✅ Retrieved {len(states)} aircraft positions")
        
        # Parse flight states into structured format
        flights = []
        for state in states[:500]:  # Limit for demo (API returns thousands)
            if state[1]:  # Has callsign
                flight = {
                    'icao24': state[0],
                    'callsign': state[1].strip() if state[1] else None,
                    'origin_country': state[2],
                    'longitude': float(state[5]) if state[5] else None,
                    'latitude': float(state[6]) if state[6] else None,
                    'baro_altitude': float(state[7]) if state[7] else None,
                    'on_ground': bool(state[8]),
                    'velocity': float(state[9]) if state[9] else None,
                    'true_track': float(state[10]) if state[10] else None,
                    'vertical_rate': float(state[11]) if state[11] else None,
                    'geo_altitude': float(state[13]) if state[13] else None,
                    'squawk': state[14],
                    'position_source': int(state[16]) if state[16] else 0,
                    'timestamp': timestamp,
                    'extraction_time': datetime.now().isoformat()
                }
                flights.append(flight)
        
        return flights, timestamp
        
    except Exception as e:
        print(f"❌ API Error: {str(e)}")
        return [], None

# Execute extraction
raw_flights, api_timestamp = extract_flight_states()

if raw_flights:
    print(f"\n📋 Sample flight data:")
    print(json.dumps(raw_flights[0], indent=2))

In [ ]:
# Cell 6: Generate Simulated Historical Flight Data
# (Supplements API data for comprehensive warehouse demo)

import random

def generate_historical_flights(num_records=1000):
    """Generate realistic historical flight data for warehouse demo"""
    
    airlines = {
        'AA': 'American Airlines', 'UA': 'United Airlines', 'DL': 'Delta Air Lines',
        'BA': 'British Airways', 'LH': 'Lufthansa', 'AF': 'Air France',
        'EK': 'Emirates', 'SQ': 'Singapore Airlines', 'QF': 'Qantas',
        'JL': 'Japan Airlines', 'CX': 'Cathay Pacific', 'NH': 'All Nippon Airways'
    }
    
    aircraft_types = ['B737', 'B777', 'B787', 'A320', 'A350', 'A380']
    airport_codes = list(AIRPORTS.keys())
    
    flights = []
    base_date = datetime.now() - timedelta(days=30)
    
    for i in range(num_records):
        airline_code = random.choice(list(airlines.keys()))
        flight_num = f"{airline_code}{random.randint(100, 9999)}"
        
        origin = random.choice(airport_codes)
        dest = random.choice([a for a in airport_codes if a != origin])
        
        # Scheduled times
        sched_dep = base_date + timedelta(
            days=random.randint(0, 30),
            hours=random.randint(0, 23),
            minutes=random.choice([0, 15, 30, 45])
        )
        
        # Flight duration based on distance (simplified)
        flight_duration = random.randint(60, 840)  # 1-14 hours in minutes
        sched_arr = sched_dep + timedelta(minutes=flight_duration)
        
        # Actual times with delays
        dep_delay = random.choices(
            [0, random.randint(1, 15), random.randint(16, 60), random.randint(61, 180)],
            weights=[0.6, 0.25, 0.1, 0.05]
        )[0]
        arr_delay = dep_delay + random.randint(-10, 20)
        
        actual_dep = sched_dep + timedelta(minutes=dep_delay)
        actual_arr = sched_arr + timedelta(minutes=max(0, arr_delay))
        
        # Flight status
        if arr_delay <= 0:
            status = 'On Time'
        elif arr_delay <= 15:
            status = 'Slight Delay'
        elif arr_delay <= 60:
            status = 'Delayed'
        else:
            status = 'Significantly Delayed'
        
        flight = {
            'flight_id': f"FL{i+1:06d}",
            'flight_number': flight_num,
            'airline_code': airline_code,
            'airline_name': airlines[airline_code],
            'aircraft_type': random.choice(aircraft_types),
            'origin_airport': origin,
            'destination_airport': dest,
            'scheduled_departure': sched_dep.isoformat(),
            'scheduled_arrival': sched_arr.isoformat(),
            'actual_departure': actual_dep.isoformat(),
            'actual_arrival': actual_arr.isoformat(),
            'departure_delay_minutes': dep_delay,
            'arrival_delay_minutes': max(0, arr_delay),
            'flight_status': status,
            'flight_duration_minutes': flight_duration,
            'passengers': random.randint(50, 400),
            'load_factor': round(random.uniform(0.55, 0.98), 2),
            'cancelled': random.random() < 0.02,  # 2% cancellation rate
            'diverted': random.random() < 0.01    # 1% diversion rate
        }
        flights.append(flight)
    
    return flights

# Generate historical data
historical_flights = generate_historical_flights(1000)
print(f"✅ Generated {len(historical_flights)} historical flight records")
print(f"\n📋 Sample historical flight:")
print(json.dumps(historical_flights[0], indent=2))

In [ ]:
# Cell 7: LOAD - Create Raw Layer DataFrames and Register as SQL Tables
# =========================================================================
# This is the RAW layer - data as extracted, minimal transformations

# Create airport dimension DataFrame
airport_data = [
    {'airport_code': code, **details} 
    for code, details in AIRPORTS.items()
]
df_airports_raw = spark.createDataFrame(airport_data)

# Create flights DataFrame
df_flights_raw = spark.createDataFrame(historical_flights)

# Create live positions DataFrame (if API call succeeded)
if raw_flights:
    df_positions_raw = spark.createDataFrame(raw_flights)
else:
    # Create empty DataFrame with schema
    df_positions_raw = spark.createDataFrame([], schema=StructType([
        StructField("icao24", StringType()),
        StructField("callsign", StringType()),
        StructField("origin_country", StringType()),
        StructField("latitude", DoubleType()),
        StructField("longitude", DoubleType())
    ]))

# ⭐ REGISTER AS SQL TEMPORARY VIEWS - This enables SQL queries!
df_airports_raw.createOrReplaceTempView("raw_airports")
df_flights_raw.createOrReplaceTempView("raw_flights")
df_positions_raw.createOrReplaceTempView("raw_positions")

print("=" * 60)
print("📦 RAW LAYER - Data Loaded and Registered as SQL Tables")
print("=" * 60)
print(f"\n✅ raw_airports: {df_airports_raw.count()} records")
print(f"✅ raw_flights: {df_flights_raw.count()} records")
print(f"✅ raw_positions: {df_positions_raw.count()} records")
print("\n🔧 Tables available for SQL queries:")
print("   - raw_airports")
print("   - raw_flights")
print("   - raw_positions")

## 🔷 SQL Section Begins Here
From this point forward, we'll heavily use **Spark SQL** for data transformations and analytics.
This mirrors real-world data engineering where SQL is the primary language for:
- Data transformations (ETL/ELT)
- Creating dimension and fact tables
- Business logic implementation
- Analytics and reporting

In [ ]:
# Cell 8: SQL - Explore Raw Data Layer
# =========================================================================
# Using SQL to understand our raw data before transformation

print("=" * 60)
print("🔍 EXPLORING RAW DATA WITH SQL")
print("=" * 60)

# Query 1: Preview raw flights
print("\n📋 Query 1: Preview Raw Flights Table")
spark.sql("""
    SELECT 
        flight_number,
        airline_name,
        origin_airport,
        destination_airport,
        flight_status,
        departure_delay_minutes
    FROM raw_flights
    LIMIT 10
""").show(truncate=False)

# Query 2: Check data quality - null counts
print("\n📋 Query 2: Data Quality Check - Null Values")
spark.sql("""
    SELECT 
        COUNT(*) as total_records,
        SUM(CASE WHEN flight_number IS NULL THEN 1 ELSE 0 END) as null_flight_numbers,
        SUM(CASE WHEN origin_airport IS NULL THEN 1 ELSE 0 END) as null_origins,
        SUM(CASE WHEN destination_airport IS NULL THEN 1 ELSE 0 END) as null_destinations,
        SUM(CASE WHEN cancelled = true THEN 1 ELSE 0 END) as cancelled_flights
    FROM raw_flights
""").show()

# Query 3: Preview airports
print("\n📋 Query 3: Airport Reference Data")
spark.sql("""
    SELECT 
        airport_code,
        name,
        city,
        country,
        ROUND(lat, 2) as latitude,
        ROUND(lon, 2) as longitude
    FROM raw_airports
    ORDER BY country, city
""").show(truncate=False)

In [ ]:
# Cell 9: SQL - Create CURATED LAYER (Dimension Tables)
# =========================================================================
# Transform raw data into clean, business-ready dimension tables

print("=" * 60)
print("🏗️ CREATING CURATED LAYER - DIMENSION TABLES (SQL)")
print("=" * 60)

# DIM_AIRPORT - Airport Dimension
spark.sql("""
    CREATE OR REPLACE TEMP VIEW dim_airport AS
    SELECT 
        ROW_NUMBER() OVER (ORDER BY airport_code) as airport_key,
        airport_code,
        name as airport_name,
        city,
        country,
        lat as latitude,
        lon as longitude,
        CASE 
            WHEN country IN ('US', 'CA', 'MX') THEN 'North America'
            WHEN country IN ('UK', 'FR', 'DE') THEN 'Europe'
            WHEN country IN ('JP', 'SG', 'HK') THEN 'Asia Pacific'
            WHEN country IN ('AE') THEN 'Middle East'
            WHEN country IN ('AU') THEN 'Oceania'
            ELSE 'Other'
        END as region,
        CURRENT_TIMESTAMP() as created_at,
        CURRENT_TIMESTAMP() as updated_at
    FROM raw_airports
""")

print("\n✅ dim_airport created:")
spark.sql("SELECT * FROM dim_airport").show(truncate=False)

# DIM_AIRLINE - Airline Dimension (extracted from flights)
spark.sql("""
    CREATE OR REPLACE TEMP VIEW dim_airline AS
    SELECT DISTINCT
        ROW_NUMBER() OVER (ORDER BY airline_code) as airline_key,
        airline_code,
        airline_name,
        CASE 
            WHEN airline_code IN ('AA', 'UA', 'DL') THEN 'US Carrier'
            WHEN airline_code IN ('BA', 'LH', 'AF') THEN 'European Carrier'
            WHEN airline_code IN ('EK') THEN 'Middle East Carrier'
            WHEN airline_code IN ('SQ', 'CX', 'JL', 'NH') THEN 'Asian Carrier'
            WHEN airline_code IN ('QF') THEN 'Oceania Carrier'
            ELSE 'Other'
        END as carrier_type,
        CURRENT_TIMESTAMP() as created_at
    FROM raw_flights
""")

print("\n✅ dim_airline created:")
spark.sql("SELECT * FROM dim_airline").show(truncate=False)

# DIM_AIRCRAFT - Aircraft Dimension
spark.sql("""
    CREATE OR REPLACE TEMP VIEW dim_aircraft AS
    SELECT DISTINCT
        ROW_NUMBER() OVER (ORDER BY aircraft_type) as aircraft_key,
        aircraft_type,
        CASE 
            WHEN aircraft_type LIKE 'B%' THEN 'Boeing'
            WHEN aircraft_type LIKE 'A%' THEN 'Airbus'
            ELSE 'Other'
        END as manufacturer,
        CASE 
            WHEN aircraft_type IN ('A320', 'B737') THEN 'Narrow Body'
            WHEN aircraft_type IN ('A350', 'B777', 'B787') THEN 'Wide Body'
            WHEN aircraft_type IN ('A380') THEN 'Super Jumbo'
            ELSE 'Unknown'
        END as aircraft_category,
        CASE 
            WHEN aircraft_type IN ('A320', 'B737') THEN 180
            WHEN aircraft_type IN ('A350') THEN 300
            WHEN aircraft_type IN ('B777') THEN 350
            WHEN aircraft_type IN ('B787') THEN 290
            WHEN aircraft_type IN ('A380') THEN 500
            ELSE 200
        END as typical_capacity
    FROM raw_flights
""")

print("\n✅ dim_aircraft created:")
spark.sql("SELECT * FROM dim_aircraft").show()

In [ ]:
# Cell 10: SQL - Create DIM_DATE (Date Dimension - Critical for any Data Warehouse!)
# =========================================================================

spark.sql("""
    CREATE OR REPLACE TEMP VIEW dim_date AS
    WITH date_range AS (
        SELECT DISTINCT 
            DATE(scheduled_departure) as flight_date
        FROM raw_flights
    )
    SELECT 
        ROW_NUMBER() OVER (ORDER BY flight_date) as date_key,
        flight_date as full_date,
        YEAR(flight_date) as year,
        MONTH(flight_date) as month,
        DAY(flight_date) as day,
        QUARTER(flight_date) as quarter,
        WEEKOFYEAR(flight_date) as week_of_year,
        DAYOFWEEK(flight_date) as day_of_week,
        DATE_FORMAT(flight_date, 'EEEE') as day_name,
        DATE_FORMAT(flight_date, 'MMMM') as month_name,
        CASE 
            WHEN DAYOFWEEK(flight_date) IN (1, 7) THEN TRUE 
            ELSE FALSE 
        END as is_weekend,
        CASE 
            WHEN MONTH(flight_date) IN (12, 1, 2) THEN 'Winter'
            WHEN MONTH(flight_date) IN (3, 4, 5) THEN 'Spring'
            WHEN MONTH(flight_date) IN (6, 7, 8) THEN 'Summer'
            ELSE 'Fall'
        END as season
    FROM date_range
    ORDER BY flight_date
""")

print("=" * 60)
print("📅 DIM_DATE - Date Dimension Created")
print("=" * 60)
spark.sql("SELECT * FROM dim_date LIMIT 10").show()

# DIM_TIME - Time Dimension
spark.sql("""
    CREATE OR REPLACE TEMP VIEW dim_time AS
    SELECT 
        hour as time_key,
        hour,
        CASE 
            WHEN hour = 0 THEN 12
            WHEN hour <= 12 THEN hour
            ELSE hour - 12
        END as hour_12,
        CASE WHEN hour < 12 THEN 'AM' ELSE 'PM' END as am_pm,
        CASE 
            WHEN hour BETWEEN 5 AND 11 THEN 'Morning'
            WHEN hour BETWEEN 12 AND 16 THEN 'Afternoon'
            WHEN hour BETWEEN 17 AND 20 THEN 'Evening'
            ELSE 'Night'
        END as time_of_day,
        CASE 
            WHEN hour BETWEEN 6 AND 9 THEN TRUE
            WHEN hour BETWEEN 17 AND 19 THEN TRUE
            ELSE FALSE
        END as is_peak_hour
    FROM (SELECT DISTINCT HOUR(scheduled_departure) as hour FROM raw_flights)
""")

print("\n⏰ DIM_TIME - Time Dimension Created")
spark.sql("SELECT * FROM dim_time ORDER BY hour").show()

In [ ]:
# Cell 11: SQL - Create FACT Tables (Star Schema Core)
# =========================================================================

print("=" * 60)
print("⭐ CREATING FACT TABLES - Star Schema Core")
print("=" * 60)

# FACT_FLIGHTS - Main fact table
spark.sql("""
    CREATE OR REPLACE TEMP VIEW fact_flights AS
    SELECT 
        f.flight_id as flight_key,
        al.airline_key,
        ac.aircraft_key,
        orig.airport_key as origin_airport_key,
        dest.airport_key as destination_airport_key,
        d.date_key as departure_date_key,
        t.time_key as departure_time_key,
        
        -- Measures (Facts)
        f.departure_delay_minutes,
        f.arrival_delay_minutes,
        f.flight_duration_minutes,
        f.passengers,
        f.load_factor,
        
        -- Degenerate dimensions
        f.flight_number,
        f.flight_status,
        f.cancelled,
        f.diverted,
        
        -- Timestamps
        f.scheduled_departure,
        f.scheduled_arrival,
        f.actual_departure,
        f.actual_arrival
        
    FROM raw_flights f
    LEFT JOIN dim_airline al ON f.airline_code = al.airline_code
    LEFT JOIN dim_aircraft ac ON f.aircraft_type = ac.aircraft_type
    LEFT JOIN dim_airport orig ON f.origin_airport = orig.airport_code
    LEFT JOIN dim_airport dest ON f.destination_airport = dest.airport_code
    LEFT JOIN dim_date d ON DATE(f.scheduled_departure) = d.full_date
    LEFT JOIN dim_time t ON HOUR(f.scheduled_departure) = t.hour
""")

print("\n✅ fact_flights created")
spark.sql("""
    SELECT 
        flight_key, airline_key, origin_airport_key, destination_airport_key,
        departure_delay_minutes, flight_status
    FROM fact_flights 
    LIMIT 10
""").show()

# FACT_DELAYS - Aggregated delay fact table
spark.sql("""
    CREATE OR REPLACE TEMP VIEW fact_delays AS
    SELECT 
        d.date_key,
        al.airline_key,
        orig.airport_key as airport_key,
        
        -- Delay metrics
        COUNT(*) as total_flights,
        SUM(CASE WHEN f.departure_delay_minutes > 0 THEN 1 ELSE 0 END) as delayed_flights,
        SUM(CASE WHEN f.departure_delay_minutes > 15 THEN 1 ELSE 0 END) as significantly_delayed,
        ROUND(AVG(f.departure_delay_minutes), 2) as avg_delay_minutes,
        MAX(f.departure_delay_minutes) as max_delay_minutes,
        SUM(CASE WHEN f.cancelled THEN 1 ELSE 0 END) as cancelled_flights,
        
        -- On-time performance
        ROUND(
            SUM(CASE WHEN f.arrival_delay_minutes <= 15 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 
            2
        ) as on_time_performance_pct
        
    FROM raw_flights f
    LEFT JOIN dim_airline al ON f.airline_code = al.airline_code
    LEFT JOIN dim_airport orig ON f.origin_airport = orig.airport_code
    LEFT JOIN dim_date d ON DATE(f.scheduled_departure) = d.full_date
    GROUP BY d.date_key, al.airline_key, orig.airport_key
""")

print("\n✅ fact_delays created")
spark.sql("SELECT * FROM fact_delays LIMIT 10").show()

In [ ]:
# Cell 12: SQL - Create FACT_ROUTES Table
# =========================================================================

spark.sql("""
    CREATE OR REPLACE TEMP VIEW fact_routes AS
    SELECT 
        orig.airport_key as origin_airport_key,
        dest.airport_key as destination_airport_key,
        al.airline_key,
        
        -- Route metrics
        COUNT(*) as total_flights,
        COUNT(DISTINCT f.flight_number) as unique_flight_numbers,
        ROUND(AVG(f.flight_duration_minutes), 0) as avg_flight_duration,
        ROUND(AVG(f.passengers), 0) as avg_passengers,
        ROUND(AVG(f.load_factor) * 100, 1) as avg_load_factor_pct,
        
        -- Delay analysis by route
        ROUND(AVG(f.departure_delay_minutes), 2) as avg_departure_delay,
        ROUND(AVG(f.arrival_delay_minutes), 2) as avg_arrival_delay,
        
        -- Performance
        ROUND(
            SUM(CASE WHEN f.arrival_delay_minutes <= 15 THEN 1 ELSE 0 END) * 100.0 / COUNT(*),
            2
        ) as route_on_time_pct,
        
        -- Reliability
        SUM(CASE WHEN f.cancelled THEN 1 ELSE 0 END) as cancelled_flights,
        SUM(CASE WHEN f.diverted THEN 1 ELSE 0 END) as diverted_flights
        
    FROM raw_flights f
    LEFT JOIN dim_airline al ON f.airline_code = al.airline_code
    LEFT JOIN dim_airport orig ON f.origin_airport = orig.airport_code
    LEFT JOIN dim_airport dest ON f.destination_airport = dest.airport_code
    GROUP BY orig.airport_key, dest.airport_key, al.airline_key
""")

print("=" * 60)
print("🛫 FACT_ROUTES - Route Analysis Fact Table")
print("=" * 60)
spark.sql("""
    SELECT 
        origin_airport_key,
        destination_airport_key,
        total_flights,
        avg_flight_duration,
        route_on_time_pct
    FROM fact_routes 
    ORDER BY total_flights DESC
    LIMIT 15
""").show()

In [ ]:
# Cell 13: SQL Analytics - On-Time Performance Analysis
# =========================================================================

print("=" * 60)
print("📊 SQL ANALYTICS - ON-TIME PERFORMANCE")
print("=" * 60)

# Query: Airline On-Time Performance Ranking
print("\n🏆 Airline On-Time Performance Ranking:")
spark.sql("""
    SELECT 
        al.airline_name,
        al.carrier_type,
        COUNT(*) as total_flights,
        SUM(CASE WHEN ff.arrival_delay_minutes <= 15 THEN 1 ELSE 0 END) as on_time_flights,
        ROUND(
            SUM(CASE WHEN ff.arrival_delay_minutes <= 15 THEN 1 ELSE 0 END) * 100.0 / COUNT(*),
            2
        ) as on_time_pct,
        ROUND(AVG(ff.departure_delay_minutes), 2) as avg_delay_mins,
        SUM(CASE WHEN ff.cancelled THEN 1 ELSE 0 END) as cancellations
    FROM fact_flights ff
    JOIN dim_airline al ON ff.airline_key = al.airline_key
    GROUP BY al.airline_name, al.carrier_type
    ORDER BY on_time_pct DESC
""").show(truncate=False)

# Query: Airport Congestion Analysis
print("\n🛬 Airport Departure Performance:")
spark.sql("""
    SELECT 
        ap.airport_name,
        ap.city,
        ap.region,
        COUNT(*) as departures,
        ROUND(AVG(ff.departure_delay_minutes), 2) as avg_delay,
        MAX(ff.departure_delay_minutes) as max_delay,
        ROUND(
            SUM(CASE WHEN ff.departure_delay_minutes > 30 THEN 1 ELSE 0 END) * 100.0 / COUNT(*),
            1
        ) as severe_delay_pct
    FROM fact_flights ff
    JOIN dim_airport ap ON ff.origin_airport_key = ap.airport_key
    GROUP BY ap.airport_name, ap.city, ap.region
    ORDER BY avg_delay DESC
""").show(truncate=False)

In [ ]:
# Cell 14: SQL Analytics - Route Congestion & Utilization
# =========================================================================

print("=" * 60)
print("📊 SQL ANALYTICS - ROUTE CONGESTION & AIRCRAFT UTILIZATION")
print("=" * 60)

# Query: Busiest Routes with full details
print("\n✈️ Top 10 Busiest Routes:")
spark.sql("""
    SELECT 
        orig.city as origin_city,
        dest.city as destination_city,
        al.airline_name,
        fr.total_flights,
        fr.avg_flight_duration as duration_mins,
        fr.avg_passengers,
        fr.avg_load_factor_pct as load_factor,
        fr.route_on_time_pct as on_time_pct
    FROM fact_routes fr
    JOIN dim_airport orig ON fr.origin_airport_key = orig.airport_key
    JOIN dim_airport dest ON fr.destination_airport_key = dest.airport_key
    JOIN dim_airline al ON fr.airline_key = al.airline_key
    ORDER BY fr.total_flights DESC
    LIMIT 10
""").show(truncate=False)

# Query: Aircraft Utilization by Type
print("\n🛩️ Aircraft Utilization Analysis:")
spark.sql("""
    SELECT 
        ac.aircraft_type,
        ac.manufacturer,
        ac.aircraft_category,
        ac.typical_capacity,
        COUNT(*) as flights_operated,
        ROUND(AVG(ff.passengers), 0) as avg_passengers,
        ROUND(AVG(ff.load_factor) * 100, 1) as avg_load_factor,
        ROUND(AVG(ff.flight_duration_minutes), 0) as avg_flight_time,
        ROUND(SUM(ff.flight_duration_minutes) / 60.0, 0) as total_flight_hours
    FROM fact_flights ff
    JOIN dim_aircraft ac ON ff.aircraft_key = ac.aircraft_key
    GROUP BY ac.aircraft_type, ac.manufacturer, ac.aircraft_category, ac.typical_capacity
    ORDER BY flights_operated DESC
""").show(truncate=False)

In [ ]:
# Cell 15: SQL Analytics - Advanced Window Functions
# =========================================================================
# Window functions are a KEY SQL skill for data engineering!

print("=" * 60)
print("📊 ADVANCED SQL - WINDOW FUNCTIONS")
print("=" * 60)

# Query: Running totals and rankings
print("\n📈 Flight Volume Trends with Running Totals:")
spark.sql("""
    SELECT 
        d.full_date,
        d.day_name,
        COUNT(*) as daily_flights,
        SUM(COUNT(*)) OVER (ORDER BY d.full_date) as cumulative_flights,
        ROUND(AVG(COUNT(*)) OVER (
            ORDER BY d.full_date 
            ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
        ), 0) as rolling_7day_avg
    FROM fact_flights ff
    JOIN dim_date d ON ff.departure_date_key = d.date_key
    GROUP BY d.full_date, d.day_name
    ORDER BY d.full_date
    LIMIT 15
""").show()

# Query: Airline performance ranking within regions
print("\n🏅 Airline Rankings by Region (PARTITION BY):")
spark.sql("""
    SELECT 
        region,
        airline_name,
        total_flights,
        on_time_pct,
        RANK() OVER (PARTITION BY region ORDER BY on_time_pct DESC) as regional_rank,
        DENSE_RANK() OVER (ORDER BY on_time_pct DESC) as global_rank
    FROM (
        SELECT 
            ap.region,
            al.airline_name,
            COUNT(*) as total_flights,
            ROUND(
                SUM(CASE WHEN ff.arrival_delay_minutes <= 15 THEN 1 ELSE 0 END) * 100.0 / COUNT(*),
                2
            ) as on_time_pct
        FROM fact_flights ff
        JOIN dim_airline al ON ff.airline_key = al.airline_key
        JOIN dim_airport ap ON ff.origin_airport_key = ap.airport_key
        GROUP BY ap.region, al.airline_name
    )
    ORDER BY region, regional_rank
""").show(truncate=False)

In [ ]:
# Cell 16: SQL Analytics - Time-Based Analysis (Peak Hours)
# =========================================================================

print("=" * 60)
print("⏰ SQL ANALYTICS - TIME-BASED PATTERNS")
print("=" * 60)

# Query: Peak hour analysis
print("\n📊 Flight Volume by Time of Day:")
spark.sql("""
    SELECT 
        t.time_of_day,
        t.is_peak_hour,
        COUNT(*) as total_flights,
        ROUND(AVG(ff.departure_delay_minutes), 2) as avg_delay,
        ROUND(
            SUM(CASE WHEN ff.arrival_delay_minutes <= 15 THEN 1 ELSE 0 END) * 100.0 / COUNT(*),
            2
        ) as on_time_pct
    FROM fact_flights ff
    JOIN dim_time t ON ff.departure_time_key = t.time_key
    GROUP BY t.time_of_day, t.is_peak_hour
    ORDER BY 
        CASE t.time_of_day 
            WHEN 'Morning' THEN 1 
            WHEN 'Afternoon' THEN 2 
            WHEN 'Evening' THEN 3 
            ELSE 4 
        END
""").show()

# Query: Weekend vs Weekday performance
print("\n📅 Weekend vs Weekday Performance:")
spark.sql("""
    SELECT 
        CASE WHEN d.is_weekend THEN 'Weekend' ELSE 'Weekday' END as day_type,
        d.day_name,
        COUNT(*) as flights,
        ROUND(AVG(ff.departure_delay_minutes), 2) as avg_delay,
        ROUND(AVG(ff.load_factor) * 100, 1) as avg_load_factor,
        ROUND(
            SUM(CASE WHEN ff.arrival_delay_minutes <= 15 THEN 1 ELSE 0 END) * 100.0 / COUNT(*),
            2
        ) as on_time_pct
    FROM fact_flights ff
    JOIN dim_date d ON ff.departure_date_key = d.date_key
    GROUP BY d.is_weekend, d.day_name, d.day_of_week
    ORDER BY d.day_of_week
""").show()

In [ ]:
# Cell 17: SQL - Create KPI Summary Views (Business Intelligence Layer)
# =========================================================================

print("=" * 60)
print("📊 CREATING KPI SUMMARY VIEWS - BI LAYER")
print("=" * 60)

# Executive Dashboard KPIs
spark.sql("""
    CREATE OR REPLACE TEMP VIEW kpi_executive_summary AS
    SELECT 
        -- Volume metrics
        COUNT(*) as total_flights,
        COUNT(DISTINCT airline_key) as active_airlines,
        COUNT(DISTINCT origin_airport_key) as active_airports,
        
        -- Performance metrics
        ROUND(AVG(departure_delay_minutes), 2) as avg_departure_delay,
        ROUND(AVG(arrival_delay_minutes), 2) as avg_arrival_delay,
        ROUND(
            SUM(CASE WHEN arrival_delay_minutes <= 15 THEN 1 ELSE 0 END) * 100.0 / COUNT(*),
            2
        ) as overall_on_time_pct,
        
        -- Capacity metrics
        SUM(passengers) as total_passengers,
        ROUND(AVG(load_factor) * 100, 1) as avg_load_factor,
        
        -- Reliability metrics
        SUM(CASE WHEN cancelled THEN 1 ELSE 0 END) as total_cancellations,
        ROUND(SUM(CASE WHEN cancelled THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as cancellation_rate,
        
        -- Operational hours
        ROUND(SUM(flight_duration_minutes) / 60.0, 0) as total_flight_hours
        
    FROM fact_flights
""")

print("\n🎯 Executive Summary KPIs:")
spark.sql("SELECT * FROM kpi_executive_summary").show(vertical=True)

# Airline Scorecard
spark.sql("""
    CREATE OR REPLACE TEMP VIEW kpi_airline_scorecard AS
    SELECT 
        al.airline_name,
        al.carrier_type,
        COUNT(*) as flights,
        ROUND(AVG(ff.departure_delay_minutes), 1) as avg_delay,
        ROUND(
            SUM(CASE WHEN ff.arrival_delay_minutes <= 15 THEN 1 ELSE 0 END) * 100.0 / COUNT(*),
            1
        ) as on_time_pct,
        ROUND(AVG(ff.load_factor) * 100, 1) as load_factor,
        SUM(ff.passengers) as passengers,
        ROUND(SUM(CASE WHEN ff.cancelled THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as cancel_rate
    FROM fact_flights ff
    JOIN dim_airline al ON ff.airline_key = al.airline_key
    GROUP BY al.airline_name, al.carrier_type
""")

print("\n🏢 Airline Scorecard:")
spark.sql("SELECT * FROM kpi_airline_scorecard ORDER BY on_time_pct DESC").show(truncate=False)

In [ ]:
# Cell 18: Visualization - Airline Performance Dashboard
# =========================================================================

print("=" * 60)
print("📊 VISUALIZATIONS")
print("=" * 60)

# Get data using SQL and convert to Pandas for visualization
airline_perf = spark.sql("""
    SELECT * FROM kpi_airline_scorecard ORDER BY on_time_pct DESC
""").toPandas()

# Create dashboard
fig1 = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'On-Time Performance by Airline',
        'Average Delay by Airline',
        'Load Factor by Airline',
        'Flights by Carrier Type'
    ),
    specs=[[{'type': 'bar'}, {'type': 'bar'}],
           [{'type': 'bar'}, {'type': 'pie'}]]
)

# On-Time Performance
fig1.add_trace(
    go.Bar(
        x=airline_perf['airline_name'],
        y=airline_perf['on_time_pct'],
        marker_color='green',
        name='On-Time %'
    ),
    row=1, col=1
)

# Average Delay
fig1.add_trace(
    go.Bar(
        x=airline_perf['airline_name'],
        y=airline_perf['avg_delay'],
        marker_color='red',
        name='Avg Delay'
    ),
    row=1, col=2
)

# Load Factor
fig1.add_trace(
    go.Bar(
        x=airline_perf['airline_name'],
        y=airline_perf['load_factor'],
        marker_color='blue',
        name='Load Factor'
    ),
    row=2, col=1
)

# Carrier Type Distribution
carrier_counts = airline_perf.groupby('carrier_type')['flights'].sum()
fig1.add_trace(
    go.Pie(
        labels=carrier_counts.index,
        values=carrier_counts.values,
        name='Carrier Type'
    ),
    row=2, col=2
)

fig1.update_layout(
    height=700,
    title_text="✈️ Airline Performance Dashboard",
    showlegend=False
)
fig1.update_xaxes(tickangle=45)
fig1.show()

In [ ]:
# Cell 19: Visualization - Route Map and Airport Analysis
# =========================================================================

# Get airport performance data
airport_perf = spark.sql("""
    SELECT 
        ap.airport_name,
        ap.city,
        ap.country,
        ap.latitude,
        ap.longitude,
        ap.region,
        COUNT(*) as total_flights,
        ROUND(AVG(ff.departure_delay_minutes), 2) as avg_delay,
        ROUND(
            SUM(CASE WHEN ff.arrival_delay_minutes <= 15 THEN 1 ELSE 0 END) * 100.0 / COUNT(*),
            1
        ) as on_time_pct
    FROM fact_flights ff
    JOIN dim_airport ap ON ff.origin_airport_key = ap.airport_key
    GROUP BY ap.airport_name, ap.city, ap.country, ap.latitude, ap.longitude, ap.region
""").toPandas()

# Global Airport Map
fig2 = px.scatter_geo(
    airport_perf,
    lat='latitude',
    lon='longitude',
    hover_name='city',
    hover_data={
        'airport_name': True,
        'total_flights': True,
        'avg_delay': ':.1f',
        'on_time_pct': ':.1f',
        'latitude': False,
        'longitude': False
    },
    color='on_time_pct',
    size='total_flights',
    color_continuous_scale='RdYlGn',
    title='🌍 Global Airport Performance Map',
    size_max=30
)

fig2.update_layout(
    geo=dict(
        showland=True,
        landcolor='lightgray',
        showocean=True,
        oceancolor='lightblue'
    ),
    height=500
)
fig2.show()

# Delay by Region
fig3 = px.bar(
    airport_perf.groupby('region').agg({
        'total_flights': 'sum',
        'avg_delay': 'mean',
        'on_time_pct': 'mean'
    }).reset_index(),
    x='region',
    y='avg_delay',
    color='on_time_pct',
    color_continuous_scale='RdYlGn',
    title='📊 Average Delay by Region',
    labels={'avg_delay': 'Avg Delay (mins)', 'on_time_pct': 'On-Time %'}
)
fig3.show()

In [ ]:
# Cell 20: Export Data Warehouse to CSV Files
# =========================================================================

print("=" * 60)
print("💾 EXPORTING DATA WAREHOUSE TO FILES")
print("=" * 60)

# Export all dimension tables
tables_to_export = [
    ('dim_airport', 'dim_airport.csv'),
    ('dim_airline', 'dim_airline.csv'),
    ('dim_aircraft', 'dim_aircraft.csv'),
    ('dim_date', 'dim_date.csv'),
    ('dim_time', 'dim_time.csv'),
    ('fact_flights', 'fact_flights.csv'),
    ('fact_delays', 'fact_delays.csv'),
    ('fact_routes', 'fact_routes.csv'),
    ('kpi_executive_summary', 'kpi_executive_summary.csv'),
    ('kpi_airline_scorecard', 'kpi_airline_scorecard.csv')
]

for table_name, filename in tables_to_export:
    try:
        df = spark.sql(f"SELECT * FROM {table_name}").toPandas()
        df.to_csv(filename, index=False)
        print(f"✅ {table_name} → {filename} ({len(df)} rows)")
    except Exception as e:
        print(f"❌ Error exporting {table_name}: {str(e)}")

print("\n" + "=" * 60)
print("✅ Data warehouse export complete!")
print("=" * 60)

In [ ]:
# Cell 21: Pipeline Summary and Data Warehouse Schema
# =========================================================================

print("\n" + "=" * 70)
print("✈️  FLIGHT OPERATIONS DATA ENGINEERING PIPELINE - SUMMARY")
print("=" * 70)

# Show all available SQL tables
print("\n📋 DATA WAREHOUSE SCHEMA:")
print("-" * 50)
print("\n🔷 DIMENSION TABLES:")
print("   • dim_airport    - Airport reference data")
print("   • dim_airline    - Airline information")
print("   • dim_aircraft   - Aircraft types & capacity")
print("   • dim_date       - Date dimension (calendar)")
print("   • dim_time       - Time dimension (hours)")

print("\n⭐ FACT TABLES:")
print("   • fact_flights   - Individual flight records")
print("   • fact_delays    - Aggregated delay metrics")
print("   • fact_routes    - Route performance summary")

print("\n📊 KPI VIEWS:")
print("   • kpi_executive_summary  - High-level metrics")
print("   • kpi_airline_scorecard  - Airline performance")

# Get summary stats
summary = spark.sql("SELECT * FROM kpi_executive_summary").collect()[0]

print("\n" + "-" * 50)
print("📈 KEY METRICS:")
print(f"   • Total Flights: {summary['total_flights']:,}")
print(f"   • Total Passengers: {summary['total_passengers']:,}")
print(f"   • Active Airlines: {summary['active_airlines']}")
print(f"   • Active Airports: {summary['active_airports']}")
print(f"   • On-Time Performance: {summary['overall_on_time_pct']}%")
print(f"   • Average Delay: {summary['avg_departure_delay']} mins")
print(f"   • Total Flight Hours: {int(summary['total_flight_hours']):,}")

print("\n" + "-" * 50)
print("🔧 SQL SKILLS DEMONSTRATED:")
print("   ✓ CREATE VIEW / CREATE TABLE")
print("   ✓ JOIN operations (LEFT, INNER)")
print("   ✓ Aggregations (COUNT, SUM, AVG, MAX)")
print("   ✓ CASE WHEN statements")
print("   ✓ GROUP BY with multiple columns")
print("   ✓ Window Functions (RANK, PARTITION BY, ROW_NUMBER)")
print("   ✓ Subqueries and CTEs")
print("   ✓ Date/Time functions")

print("\n" + "=" * 70)
print("✅ Pipeline executed successfully!")
print("=" * 70)

## 📚 SQL Skills Summary for Data Engineering

This pipeline demonstrates **essential SQL skills** used daily in data engineering:

| SQL Concept | Usage in Pipeline | Real-World Application |
|-------------|-------------------|----------------------|
| **CREATE VIEW** | Dimension & Fact tables | Building data warehouse layers |
| **JOIN** | Connecting facts to dimensions | Star schema queries |
| **GROUP BY** | Aggregating metrics | KPI calculations |
| **Window Functions** | RANK, PARTITION BY | Running totals, rankings |
| **CASE WHEN** | Categorization logic | Business rules |
| **Subqueries** | Complex aggregations | Multi-level analytics |
| **CTEs** | Date dimension generation | Readable complex queries |

### 🎯 Why This Matters for Recruiters
- **70%+ of data engineering work is SQL-based**
- Shows understanding of **dimensional modeling** (Star Schema)
- Demonstrates **ETL pipeline thinking** (Raw → Curated → BI layers)
- Real **aviation/logistics domain knowledge**